In [44]:
import tensorflow as tf
tf.__version__

'2.10.0'

In [45]:
print(tf.test.is_built_with_cuda())
print(tf.test.is_built_with_gpu_support())
print(tf.test.gpu_device_name())

True
True
/device:GPU:0


In [46]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5140245241206699428
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10067378176
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9671753699741627042
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:2b:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [47]:
import numpy as np 
import pandas as pd 
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.applications import EfficientNetB0
import matplotlib.pyplot as plt

In [48]:
train_ds="C:\Download/Birds525/train"
test_ds="C:\Download/Birds525/test"
validation_ds="C:\Download/Birds525/valid"

In [49]:
train_datagen = ImageDataGenerator(rescale = 1./255.) #initialize train generator 
valid_datagen = ImageDataGenerator(rescale = 1./255.) #initialize validation generator 
test_datagen = ImageDataGenerator(rescale = 1./255.) #initialize test generator

In [50]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_ds,target_size=(224, 224),batch_size=32,shuffle=True,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_ds,target_size=(224,224),batch_size=32,shuffle=False,class_mode='categorical')
validation_generator = valid_datagen.flow_from_directory(validation_ds,target_size=(224,224),batch_size=32,shuffle=False,class_mode='categorical')

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [51]:
train_generator[0][0].shape

(32, 224, 224, 3)

In [52]:
print(len(train_generator))
print(len(test_generator))
print(len(validation_generator))

2645
83
83


In [53]:
model = EfficientNetB0(include_top = False, input_tensor = None, input_shape = None, pooling = None, classes =525)

In [54]:
opt = Adam(learning_rate = 0.0001)
model.compile(optimizer = opt, loss = categorical_crossentropy, metrics = ['accuracy'])

In [55]:
model.fit(train_generator, batch_size = 32, epochs = 40)

Epoch 1/40


ValueError: in user code:

    File "c:\Users\woghk\anaconda3\envs\JayTF\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\woghk\anaconda3\envs\JayTF\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\woghk\anaconda3\envs\JayTF\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\woghk\anaconda3\envs\JayTF\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\woghk\anaconda3\envs\JayTF\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\woghk\anaconda3\envs\JayTF\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
   

In [ ]:
n_categories = len(os.listdir("C:\Download/Birds525/train")) # number of categories print(n_categories)
n_categories

525

In [ ]:
results =pd.DataFrame(history.history)
results.head()

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train','Val'], loc= 'upper left')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
y_test_1 = test_generator.classes
y_pred_1 =model.predict(test_generator)
y_pred_1 = np.argmax(y_pred_1,axis=1)

results =model.evaluate(test_generator)